# 1. Importación de librerías

**Nota**: En caso de necesitar instalar las librerías se puede ejecutar el siguiente bloque antes de realizar las importaciones:

In [ ]:
!pip install opencv-python==4.10.0.82 numpy==1.23.5 mediapipe==0.10.14 tensorflow==2.12.0rc0

In [ ]:
import os
import cv2
import time
import numpy as np
import mediapipe as mp
import tensorflow as tf
from matplotlib import pyplot as plt

# Para la partición de datos de entrenamiento y prueba
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Para la creación del modelo con LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

# Para la evaluación del modelo con LSTM
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

# 2. Uso de OpenCV para detectar partes del cuerpo

## 2.1. Detección de puntos de interés (landmarks) de cara, brazos y manos con OpenCV 

In [ ]:
def body_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Se convierte porque CV2 trabaja con BGR y el modelo con RGB
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Se regresa al valor por defecto BGR de CV2

    return image, results

In [ ]:
def draw_landmarks(image, results):
    mp_draw.draw_landmarks(
        image, 
        results.face_landmarks, 
        mp_hol.FACEMESH_TESSELATION,
        mp_draw.DrawingSpec(color=(255,255,255), thickness=1, circle_radius=1),
        mp_draw.DrawingSpec(color=(255,255,255), thickness=1, circle_radius=1)
    )
    mp_draw.draw_landmarks(
        image, 
        results.pose_landmarks, 
        mp_hol.POSE_CONNECTIONS,
        mp_draw.DrawingSpec(color=(255,0,0), thickness=1, circle_radius=1),
        mp_draw.DrawingSpec(color=(255,0,0), thickness=1, circle_radius=1)
    )
    mp_draw.draw_landmarks(
        image, 
        results.left_hand_landmarks, 
        mp_hol.HAND_CONNECTIONS,
        mp_draw.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1),
        mp_draw.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1)
    )
    mp_draw.draw_landmarks(
        image, 
        results.right_hand_landmarks, 
        mp_hol.HAND_CONNECTIONS,
        mp_draw.DrawingSpec(color=(0,0,255), thickness=1, circle_radius=1),
        mp_draw.DrawingSpec(color=(0,0,255), thickness=1, circle_radius=1)
    )

## 2.2. Extracción de puntos de interés (landmarks) a un arreglo de numpy para ser utilizado por la LSTM neural network

In [ ]:
def get_landmarks_array(results):
    face = np.array([[r.x, r.y, r.z] for r in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*4) 
    pose = np.array([[r.x, r.y, r.z, r.visibility] for r in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4) 
    left_hand = np.array([[r.x, r.y, r.z] for r in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[r.x, r.y, r.z] for r in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([face, pose, left_hand, right_hand])

In [ ]:
mp_hol = mp.solutions.holistic
mp_draw = mp.solutions.drawing_utils
holistic = mp_hol.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7)

**Nota**: Este bloque de código es para verificar que los landmarks se están mostrando correctamente en la pantalla, se puede omitir en la ejecución del programa

In [ ]:
while cap.isOpened():
    # Obtiene la imagen de la camara
    ret, frame = cap.read()

    # Realiza la detección de rostro, brazos y manos con mediapipe
    image, results = body_detection(frame, holistic)
    
    # Dibuja los puntos detectados con el modelo holistico y los agrega a la captura de video
    draw_landmarks(image, results)
    
    # Muestra captura de video en pantalla
    cv2.imshow('OpenCV', image)

    # En caso de oprimir ESC, sale del While
    if cv2.waitKey(27) & 0xFF == ord('\x1b'):
        break

cap.release()
cv2.destroyAllWindows()

# 3. Generación de conjunto de datos para el entrenamiento y validación

**Nota**: las secciones 3.2. y 3.3. pueden omitirse si ya se generaron datos de entrenamiento

## 3.1. Configuración de la carpeta de datos

In [ ]:
# Variables de open CV y MediaPipe para obtener los landmarks
cap = cv2.VideoCapture(0)
mp_hol = mp.solutions.holistic
mp_draw = mp.solutions.drawing_utils
holistic = mp_hol.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Ruta de la carpeta de datos
folder_path = os.path.join('data')

# Configuracion del numero de secuencias a generar por cada gesto y la cantidad de frames a recolectar por cada secuencia
gestures = np.array(['hola','gracias'])
seq_amount = 30 # 30 secuencias por gesto
seq_length = 30 # 30 frames por secuencia

## 3.2. Creación de carpetas

In [ ]:
for g in gestures:
    for s in range(seq_amount):
        try:
            os.makedirs(os.path.join(folder_path, g, str(s)))
        except:
            pass

## 3.3. Programa para la captación de datos

In [ ]:
for g in gestures:
    for s in range(seq_amount):
        for frame_number in range(seq_length):
            
            # Obtiene la imagen de la camara
            ret, frame = cap.read()
        
            # Realiza la detección de rostro, brazos y manos con mediapipe
            image, results = body_detection(frame, holistic)
            
            # Dibuja los puntos detectados con el modelo holistico y los agrega a la captura de video
            draw_landmarks(image, results)

            # Inicia la recolección de datos de los gestos, muestra en pantalla mensaje para hacer gestos y espera 1 segundo para tomar nuevos datos
            if frame_number == 0:
                cv2.putText(image, 'Starting data gathering', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 4, cv2.LINE_AA)
                cv2.putText(image, f'Gathering frames for {g} - Video No. {s}', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
                cv2.waitKey(1000)
            else:
                cv2.putText(image, f'Gathering frames for {g} - Video No. {s}', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)

            # Guarda los datos obtenidos en un archivo
            keypoints = get_landmarks_array(results)
            output_path = os.path.join(folder_path, g, str(s), str(frame_number))
            np.save(output_path, keypoints)
            
            # Muestra captura de video en pantalla
            cv2.imshow('OpenCV', image)
        
            # En caso de oprimir ESC, sale del While
            if cv2.waitKey(27) & 0xFF == ord('\x1b'):
                break

cap.release()
cv2.destroyAllWindows()

# 4. Preprocesamiento y preparación de datos

## 4.1. Obtención de los datos generados en la sección 3. Generación de conjunto de datos para el entrenamiento y validación

In [ ]:
label_map = {label:num for num, label in enumerate(gestures)}
sequences = []
labels = []

for gesture in gestures:
    for sequence in range(seq_amount):
        window = []
        for frame_number in range(seq_length):
            res = np.load(os.path.join(folder_path, gesture, str(sequence), f"{frame_number}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[gesture])

## 4.2. Partición del conjunto de datos en datos de entrenamiento y datos de prueba

In [ ]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

# 5. Construcción del modelo con LSTM para la predicción de gestos en lengua de signos

## 5.1. Definición de los parámetros para guardar los logs en TensorBoard

In [ ]:
logs_path = os.path.join('logs')
tb_callback = TensorBoard(log_dir=logs_path)

## 5.2. Configuración del modelo con LSTM

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(gestures.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

## 5.3. Entrenamiento del modelo de predicción de gestos en lengua de signos

**Nota**: En caso de tener un modelo ya entrenado, se puede omitir esta parte y pasar a la sección 5.4. 

In [ ]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])
model.summary()
model.save('predictor_LSE.h5')

## 5.4. Carga del modelo entrenado

In [ ]:
model.load_weights('predictor_LSE.h5')

## 5.5. Evaluación del modelo entrenado

In [ ]:
pred_test = model.predict(x_test)
pred_test_real = np.argmax(y_test, axis=1).tolist()
pred_test = np.argmax(pred_test, axis=1).tolist()
print("Matriz de confusión: \n", multilabel_confusion_matrix(pred_test_real, pred_test))
print("\n Precisión: ", accuracy_score(pred_test_real, pred_test))

# 6. Pruebas de reconocimiento en tiempo real

In [ ]:
sequence = []
sentence = []
predictions = []
threshold = 0.6
cap = cv2.VideoCapture(0)

while cap.isOpened():
    # Obtiene la imagen de la camara
    ret, frame = cap.read()

    # Realiza la detección de rostro, brazos y manos con mediapipe
    image, results = body_detection(frame, holistic)
    
    # Dibuja los puntos detectados con el modelo holistico y los agrega a la captura de video
    draw_landmarks(image, results)

    # Procesamiento de predicción
    landmarks = get_landmarks_array(results)
    sequence.append(landmarks)
    sequence = sequence[-30:]
    pred = model.predict(np.expand_dims(sequence, axis=0))[0]
    predictions.append(np.argmax(pred))

    if len(sequence) == 30:
        pred = model.predict(np.expand_dims(sequence, axis=0))[0]
        print(f'Prediction = {gestures[np.argmax(pred)]}, {pred}')
        predictions.append(np.argmax(pred))

    # Visualización de resultados
    if np.unique(predictions[-10:])[0]== np.argmax(pred):
        if pred[np.argmax(pred)] > threshold:
            if len(sentence) > 0:
                if gestures[np.argmax(pred)] != sentence[-1]:
                    sentence.append(gestures[np.argmax(pred)])
            else:
                sentence.append(gestures[np.argmax(pred)])
    
    if len(sentence) > 5:
        sentence = sentence[-5:]

    output_text = ' '.join(sentence)
    cv2.rectangle(image, (0,0), (640,40), (245,117,16), -1)
    cv2.putText(image, output_text, (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    # Muestra captura de video en pantalla
    cv2.imshow('OpenCV', image)

    # En caso de oprimir ESC, sale del While
    if cv2.waitKey(27) & 0xFF == ord('\x1b'):
        break

cap.release()
cv2.destroyAllWindows()